# Evaluation of model performance

## Preamble

In [ ]:
import hist
import matplotlib.gridspec as grid_spec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy
import uproot
from iminuit import Minuit, cost
from iminuit.cost import LeastSquares

from plotting import watermark

In [ ]:
plt.style.use(["science", "notebook"])

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

## Split model (lepton and hadron)

In [ ]:
df_lepton = pd.read_csv("CNN_3d_cerrotorre_lepton_energy_n15265_e40.csv")
df_hadron = pd.read_csv("CNN_3d_cerrotorre_hadron_energy_n15265_e40.csv")

df = pd.concat([df_hadron, df_lepton], axis=1)

In [ ]:
df["d_lepton_energy"] = df.lepton_energy_pred - df.lepton_energy_test

In [ ]:
df["d_hadron_energy"] = df.hadron_energy_pred - df.hadron_energy_test

In [ ]:
df.d_hadron_energy.hist()
df.d_lepton_energy.hist()
# TODO make 2d hist

In [ ]:
df["total_energy_pred"] = df.lepton_energy_pred + df.hadron_energy_pred
df["total_energy_test"] = df.lepton_energy_test + df.hadron_energy_test

In [ ]:
df["d_total_energy"] = df.total_energy_pred - df.total_energy_test

In [ ]:
scale_lepton = df.lepton_energy_test.mean() / df.lepton_energy_pred.mean()

In [ ]:
shift_lepton = ((scale_lepton * df.lepton_energy_pred) - df.lepton_energy_test).mean()

In [ ]:
df["corrected_lepton_energy"] = (scale_lepton * df.lepton_energy_pred) - shift_lepton

In [ ]:
scale_hadron = df.hadron_energy_test.mean() / df.hadron_energy_pred.mean()

In [ ]:
shift_hadron = ((scale_hadron * df.hadron_energy_pred) - df.hadron_energy_test).mean()

In [ ]:
df["corrected_hadron_energy"] = (scale_hadron * df.hadron_energy_pred) - shift_hadron

In [ ]:
df["corrected_total_energy"] = df.corrected_hadron_energy + df.corrected_lepton_energy

## $E_\nu$ model

### Load and prepare data

In [ ]:
model_name = "CNN_3dSat5_grandjorasses_nu_energy"

In [ ]:
df = pd.read_csv("CNN_3dSat5_grandjorasses_nu_energy_n60578_e100.csv")

In [ ]:
df["total_energy_test"] = df.nu_energy_test
df["total_energy_pred"] = df.nu_energy_pred

In [ ]:
df["corrected_total_energy"] = df.total_energy_pred

### Correct energy scale

In [ ]:
scale2 = df.total_energy_test.mean() / df.corrected_total_energy.mean()

In [ ]:
scale = np.exp(
    (np.log(df.total_energy_test) - np.log(df.corrected_total_energy)).mean()
)

In [ ]:
shift = ((scale * df.corrected_total_energy) - df.total_energy_test).mean()

In [ ]:
shift2 = ((scale2 * df.corrected_total_energy) - df.total_energy_test).mean()

In [ ]:
df["corrected_total_energy2"] = (scale2 * df.corrected_total_energy) - shift2

In [ ]:
df.corrected_total_energy = (scale * df.corrected_total_energy) - shift

In [ ]:
print(scale, scale2, shift, shift2)

Offset should be consistent with zero, small scale factor seems to be needed, consistent with small bias in $\log{E}$

In [ ]:
df["d_corrected_energy"] = df.corrected_total_energy - df.total_energy_test

In [ ]:
bins = np.linspace(0, 5000, 51)
plt.hist(df.total_energy_test, bins=bins, label="true", histtype="step")
plt.hist(df.total_energy_pred, bins=bins, label="uncorrected", alpha=0.5)
plt.hist(df.corrected_total_energy, bins=bins, label="corrected", alpha=0.5)
plt.hist(
    df.corrected_total_energy2, bins=bins, label="corrected - alternative", alpha=0.5
)
plt.xlabel(r" $E\;[\mathrm{GeV}]$")
watermark()
plt.legend()
plt.savefig(f"plots/energy_correction_{model_name}.png")
plt.savefig(f"plots/energy_correction_{model_name}.pdf")

In [ ]:
(df.d_corrected_energy / df.total_energy_test).hist(bins=100, log=True)

In [ ]:
df.d_corrected_energy.hist(bins=100, log=True)

In [ ]:
df.d_corrected_energy[(1000 < df.total_energy_test) & df.total_energy_test < 2000].hist(
    bins=100, log=True
)

### Fit energy resolution

In [ ]:
bins_E_reco = 14

In [ ]:
(2000 - 200) / 15

In [ ]:
df.total_energy_test.min()

In [ ]:
h_dE_rel_test_vs_E_rel_pred = (
    hist.Hist.new.Regular(200, -1000, 1000, name=r"dE")
    .Regular(
        bins_E_reco, 320, 2000, name=r"E_true"
    )  # , transform=hist.axis.transform.log)
    .Double()
)

In [ ]:
h_dE_rel_test_vs_E_rel_pred.fill(df.d_corrected_energy, df.total_energy_test)

In [ ]:
h_dE_rel_test_vs_E_rel_pred.plot()
plt.xlabel(r" $\Delta E\;[\mathrm{GeV}]$")
plt.ylabel(r"true $E\;[\mathrm{GeV}]$")
ax = plt.gca()
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/h_dE_rel_test_vs_E_rel_pred.pdf")
plt.savefig("plots/h_dE_rel_test_vs_E_rel_pred.png")

In [ ]:
def model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, loc=mu, scale=sigma)

In [ ]:
gs = grid_spec.GridSpec(bins_E_reco, 1)
fig = plt.figure(figsize=(16, 9))

i = 0
mus = []
sigmas = []
bins = []

ax_objs = []
for bin in range(bins_E_reco):
    # creating new axes object
    ax_objs.append(fig.add_subplot(gs[i : i + 1, 0:]))

    # plotting the distribution
    h = h_dE_rel_test_vs_E_rel_pred[:, bin]
    h.plot(yerr=False, ax=ax_objs[-1], color=colors[bin % len(colors)], histtype="fill")
    entries, edges = h.to_numpy()
    n_bins = len(entries)
    average = np.average(edges[:-1], weights=entries)
    variance = np.average((edges[:-1] - average) ** 2, weights=entries)
    m = Minuit(cost.BinnedNLL(entries, edges, model), average, np.sqrt(variance))
    res = m.migrad()
    if res.valid:
        plot_range = ax_objs[-1].get_xlim()
        x = np.linspace(*plot_range, 100)
        best_fit = scipy.stats.norm(res.params[0].value, res.params[1].value)
        binsize = (plot_range[1] - plot_range[0]) / n_bins
        scale = (
            h.sum()
            / (best_fit.cdf(plot_range[1]) - best_fit.cdf(plot_range[0]))
            * binsize
        )
        ax_objs[-1].plot(
            x, scale * best_fit.pdf(x), color=colors[(bin + 3) % len(colors)]
        )
        bins.append(bin)
        mus.append(res.params[0])
        sigmas.append(res.params[1])
    else:
        print(res)

    # make background transparent
    rect = ax_objs[-1].patch
    rect.set_alpha(0)

    # remove borders, axis ticks, and labels
    ax_objs[-1].set_yticklabels([])

    if i == bins_E_reco - 1:
        ax_objs[-1].set_xlabel(r"$\Delta E$", fontsize=16, fontweight="bold")
    else:
        ax_objs[-1].set_xticklabels([])
        ax_objs[-1].set_xlabel("")

    ax_objs[-1].set_ylabel(str(bin), rotation=45)
    ax_objs[-1].set_yticks([])
    ax_objs[-1].set_xticks([])

    spines = ["top", "right", "left", "bottom"]
    for s in spines:
        ax_objs[-1].spines[s].set_visible(False)

    i += 1

gs.update(hspace=-0.7)
# gs.update()

plt.tight_layout()
plt.show()

In [ ]:
bin_edges = h_dE_rel_test_vs_E_rel_pred[0, :].to_numpy()[1]
bin_centres = (bin_edges[1:] + bin_edges[:-1]) / 2
bin_half_widths = (bin_edges[1:] - bin_edges[:-1]) / 2

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.set_xlabel("X data")
ax1.set_ylabel("Y1 data", color="g")
ax2.set_ylabel("Y2 data", color="b")
ax1.errorbar(
    bin_centres[bins],
    [mu.value for mu in mus] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[mu.error for mu in mus] / bin_centres[bins],
    linestyle="",
    label=r"$\left<\Delta E\right>$",
    color=colors[0],
)
ax2.errorbar(
    bin_centres[bins],
    [sigma.value for sigma in sigmas] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[sigma.error for sigma in sigmas] / bin_centres[bins],
    linestyle="",
    label=r"$\sigma\left(\Delta E\right)$",
    color=colors[1],
)
# ax1.hlines(0, *plt.xlim(), color='red')
ax1.set_ylabel(r"$\frac{\left<\Delta E\right>}{E_\mathrm{true}}$", color=colors[0])
ax2.set_ylabel(
    r"$\frac{\sigma\left(\Delta E\right)}{E_\mathrm{true}}$", color=colors[1]
)
ax1.set_xlabel(r"$E_\mathrm{true}\;[\mathrm{GeV}]$")

In [ ]:
plt.errorbar(
    bin_centres[bins],
    [mu.value for mu in mus] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=[mu.error for mu in mus] / bin_centres[bins],
    linestyle="",
    label=r"$\left<\Delta E\right>$",
    color=colors[0],
)
plt.hlines(0, *plt.xlim(), color="red")
plt.ylabel(r"$\frac{\left<\Delta E\right>}{E_\mathrm{true}}$")
plt.xlabel(r"$E_\mathrm{true}\;[\mathrm{GeV}]$")
ax = plt.gca()
plt.text(
    0.0,
    1.02,
    "preliminary",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.text(
    0.8,
    1.02,
    "AdvSND",
    fontweight="bold",
    fontfamily="sans-serif",
    fontsize=16,
    transform=ax.transAxes,
    usetex=False,
)
plt.savefig("plots/energy_bias.pdf")
plt.savefig("plots/energy_bias.png")

In [ ]:
A, b, E = sympy.symbols("A b E")

In [ ]:
f = A + b / sympy.sqrt(E)

In [ ]:
f_lambda = sympy.lambdify((A, b, E), f)

In [ ]:
def E_model(E, A, b):
    return f_lambda(A, b, E)

In [ ]:
sigma_E_over_E = np.array([sigma.value for sigma in sigmas]) / bin_centres[bins]
d_sigma_E_over_E = (
    [sigma.value for sigma in sigmas]
    / bin_centres[bins]
    * np.sqrt(
        (
            np.array([sigma.error for sigma in sigmas])
            / np.array([sigma.value for sigma in sigmas])
        )
        ** 2
        + (bin_half_widths[bins] / bin_centres[bins]) ** 2
    )
)

In [ ]:
least_squares = LeastSquares(
    bin_centres[bins], sigma_E_over_E, d_sigma_E_over_E, E_model
)

In [ ]:
m = Minuit(least_squares, A=0.1, b=1)  # starting values for α and β

m.migrad()  # finds minimum of least_squares function
res = m.hesse()  # accurately computes uncertainties

In [ ]:
res

In [ ]:
f_pretty = sympy.latex(
    f.subs(
        [
            (A, sympy.Float(res.params[0].value, 1)),
            (b, sympy.Float(res.params[1].value, 2)),
        ]
    )
)

In [ ]:
plt.errorbar(
    bin_centres[bins],
    [sigma.value for sigma in sigmas] / bin_centres[bins],
    xerr=bin_half_widths[bins],
    yerr=d_sigma_E_over_E,
    linestyle="",
    label=r"$\sigma\left(\Delta E\right)$",
    color=colors[1],
    fmt="o",
    capsize=3,
)
plt.plot(
    bin_centres[bins],
    E_model(bin_centres[bins], res.params[0].value, res.params[1].value),
)
plt.ylabel(r"$\frac{\sigma\left(\Delta E\right)}{E_\mathrm{true}}$")
plt.xlabel(r"$E_\mathrm{true}\;[\mathrm{GeV}]$")
ax = plt.gca()
watermark()
plt.text(
    0.6,
    0.7,
    # rf"$\sqrt{{{res.params[0].value:.3f}^2 + \left(\frac{{{res.params[1].value:.1f}}}{{\sqrt{{E}}}}\right)^2}}$",
    rf"${f_pretty}$",
    fontsize=14,
    transform=ax.transAxes,
)
plt.text(
    0.6,
    0.6,
    rf"$A = {res.params[0].value:.2f} \pm {res.params[0].error:.2f}$",
    fontsize=14,
    transform=ax.transAxes,
)
plt.text(
    0.6,
    0.5,
    rf"$b = {res.params[1].value:.1f} \pm {res.params[1].error:.1f}$",
    fontsize=14,
    transform=ax.transAxes,
)

plt.savefig(f"plots/energy_resolution_{model_name}.pdf")
plt.savefig(f"plots/energy_resolution_{model_name}.png")

### Convergence history

In [ ]:
history_df = pd.read_csv("history_CNN_3dSat5_grandjorasses_nu_energy_n75843_e100.csv")

In [ ]:
n_events = 75843

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
# plt.title("CNN $")
ax1.plot(history_df["loss"].values, color=colors[0])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss Function", color=colors[0])
try:
    ax2.plot(history_df["mae"].values, color=colors[1])
except KeyError:
    ax2.plot(history_df["dense_2_mae"].values, color=colors[1])
    ax2.plot(history_df["dense_3_mae"].values, color=colors[1])
ax2.set_ylabel("Error", color=colors[1])
plt.text(
    0.3,
    0.7,
    f"Training dataset: {n_events} events\n"
    # f"Test dataset: {events_test.num_entries} events\n"
    f"Training duration: {len(history_df)} epochs\n{model_name}",
    transform=ax1.transAxes,
)
watermark()
plt.savefig(f"plots/convergence_{model_name}_n{n_events}_e{len(history_df)}.pdf")
plt.savefig(f"plots/convergence_{model_name}_n{n_events}_e{len(history_df)}.png")

### Distributions of $\log{E}$

In [ ]:
(np.log(df.nu_energy_pred) - np.log(df.nu_energy_test)).hist(bins=100)

In [ ]:
plt.figure()
np.log(df.nu_energy_pred).hist()
plt.figure()
np.log(df.nu_energy_test).hist()

## $z_0$

In [ ]:
df_start_z = pd.read_csv("CNN_3d_breithorn_start_z_n60578_e87.csv")

In [ ]:
df_start_z

In [ ]:
(df_start_z.start_z_pred - df_start_z.start_z_test).hist(bins=1000)
(df_start_z.start_z_pred - df_start_z.start_z_test).std()

In [ ]:
filename_test = "dataframe_CC_test.root:df"

In [ ]:
events_test = uproot.open(filename_test)

In [ ]:
events_test.keys()

In [ ]:
events_test.arrays(["start_x", "start_y", "start_z"], library="pd")